# Goal
The following is the code for scraping LinkedIn for STEM students, find the general skills and degree related.
I would use Beautiful soup and selenium together
|job title|major|skills|

# Selenium+BeautifulSoup

In [1]:
import math
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()
from warnings import warn
from datetime import datetime


In [2]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search?keywords=Physical%20Chemistry&location=United%20States&geoId=&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
no_of_jobs = 50

In [3]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome('/Users/livi/Git/Webscraper/chromedriver')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [4]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= math.ceil(no_of_jobs/25): 
    #driver.find_element_by_xpath('/html/body/main/div/section/button').send_keys(Keys.PAGE_DOWN)
    driver.execute_script("window.scrollTo(0,  document.body.scrollHeight);")
    i = i + 1
    sleep(15)

In [5]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 50 jobs.


In [6]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:

    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

In [7]:
# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)

    # job description    
    jb = driver.find_element_by_class_name('show-more-less-html')
    jb_html=jb.get_attribute('outerHTML')
    del jb
    jb = BeautifulSoup(jb_html, 'html.parser')
    jj = jb.find('div', class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5')
    #full_description='\n'.join(jj.find_all(text=True))
    full_description=jj.find_all(text=True)
    job_desc.append(full_description)


    job_criteria=driver.find_elements_by_class_name('job-criteria__item')
    for jc in job_criteria:
        func,var=jc.text.split('\n')[0],jc.text.split('\n')[1]
        if func=='Seniority level':
            level.append(var)
        if func=='Employment type':
            emp_type.append(var)
        if func=='Job function':
            functions.append(var)
        if func=='Industries':
            industries.append(var)

In [8]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

50
50
50
50
50
50
50
50
49
50


In [9]:
# creating a dataframe
job_data = pd.DataFrame.from_dict({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,'Function': functions,'Industry': industries},orient='index').T


# cleaning description column
#job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        50 non-null     object
 1   Date          50 non-null     object
 2   Company Name  50 non-null     object
 3   Post          50 non-null     object
 4   Location      50 non-null     object
 5   Description   50 non-null     object
 6   Level         50 non-null     object
 7   Type          50 non-null     object
 8   Function      49 non-null     object
 9   Industry      50 non-null     object
dtypes: object(10)
memory usage: 4.0+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2459386036,2021-03-11,CIBC Graphic,Data Scientist,"Toronto, Ontario, Canada",[We’re building a relationship-oriented bank f...,Entry level,Full-time,EngineeringInformation Technology,Information Technology and ServicesHuman Resou...
1,2335116409,2021-02-17,Wish Graphic,Data Scientist,"Toronto, Ontario, Canada","[Company Description, Wish is a mobile e-comme...",Associate,Full-time,Engineering,Computer SoftwareInternetConsumer Goods
2,2430316462,2021-02-19,Ample Insight Graphic,Data Scientist (remote),"Toronto, Ontario, Canada","[Company Description, You will join a world-cl...",Associate,Full-time,EngineeringInformation Technology,Information Technology and ServicesComputer So...
3,2377887650,2021-01-23,Yelp Graphic,Data Scientist (Remote),"Toronto, Ontario, Canada","[At Yelp, it’s our mission to connect people w...",Entry level,Full-time,EngineeringInformation Technology,Computer SoftwareInternetMarketing and Adverti...
4,2444193725,2021-03-12,OMERS Graphic,Data Scientist,"Toronto, Ontario, Canada","[Data Scientist, Head Office Toronto , Why joi...",Not Applicable,Full-time,EngineeringInformation Technology,Financial Services


In [10]:
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
na='job'+'_'.join(dt_string.split(' ')[0].split('/'))+'_'+'_'.join(dt_string.split(' ')[1].split(':'))+'.csv'
job_data.to_csv(na)